This is used for reading the test data sets, I wanted to find the number of categorical columns so i selected the int data type columns and printed them 

In [ ]:
import pandas as p
train_df=p.read_csv("train_set.csv")
test_df=p.read_csv("test_set.csv")
print("Train df head")
print(train_df.head())
print(train_df.select_dtypes(include=['int']).columns)
print(train_df['X16'].mean)
# Count the number of non-zero values in column x16
non_zero_count = (train_df['X16'] != 0).sum()
print("Number of non-zero values in x16:", non_zero_count)
unique_values = train_df['X16'].unique()
print("Unique values in X16:", unique_values)
print("Test df head")
print(test_df.head())

I checked columns separately which had integer values and had unique values between the range of 3 to 4 unique, i thought they were categorical and so i mean imputed the rest and mode imputed the integer ones

In [ ]:
from sklearn.impute import SimpleImputer
import pandas as pd

# Specify columns for different imputation strategies
mean_impute_cols = [col for col in train_df.columns if col not in ['X4', 'X5', 'X6', 'X8', 'X10', 'X11', 'X16', 'Y']]
mode_impute_cols = ['X4', 'X5', 'X6', 'X8', 'X10', 'X11', 'X16']

# Mean impute selected columns
mean_imputer = SimpleImputer(strategy='mean')
train_df[mean_impute_cols] = mean_imputer.fit_transform(train_df[mean_impute_cols])

# Mode impute the remaining specified columns
mode_imputer = SimpleImputer(strategy='most_frequent')
train_df[mode_impute_cols] = mode_imputer.fit_transform(train_df[mode_impute_cols])

print(train_df.shape)

The following code checks the correlation of columns using a correlation matrix and then filters out or drops the columns which had correlation greater than or equal to the set threshold, the columns which were dropped were appended in a list so that later on they can be dropped from the test data set too

80% correlation was the optimal one, threshold set at above 80 or below 80 didnt work that well

In [ ]:
# Block 3: Calculating Correlation and Reducing Features
import pandas as pd
import numpy as np  # Explicitly import numpy

dropped_columns = []

# Calculate the correlation matrix
correlation_matrix = train_df.corr().abs()

# Set threshold for removing correlated features before it was 0.85
correlation_threshold = 0.8

# Select upper triangle of the correlation matrix
upper_triangle = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))

# Identify features to drop based on the correlation threshold
to_drop = [column for column in upper_triangle.columns if any(upper_triangle[column] > correlation_threshold)]

# Store the names of dropped columns
dropped_columns = to_drop.copy()

# Drop the highly correlated features from the training set
train_df_reduced = train_df.drop(columns=to_drop)

# Redefine Features and Target with reduced features
Features_reduced = train_df_reduced.drop(columns=['RecordId', 'Y'])
Target = train_df_reduced['Y']

# Print the reduced feature set
print(f"Features reduced from {train_df.shape[1]} to {train_df_reduced.shape[1]}")
print(Features_reduced.head())
print(Features_reduced.shape)
print(dropped_columns)

The code below standardizes the training data set as minmax was not working well for the data set

In [ ]:
#Scaling
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
Features = scaler.fit_transform(Features_reduced)

As the estimators increase the accuracy gradually increases, I tried Gradient Boosting and before this Bagging as well, Ada performed well as i explored the base estimator a bit more for Ada. Setting the depth to less as well as the learning rate to lesser values increased the overall performance of the model. I tried giving important to positive or 1 class values by setting the class_weight to 'balanced' which did increase the recall of the model howevre at the cost of precision. The overall accuracy then stayed at 93%. 

I had to then change the approach and sadly focus on precision even though in health sets it is better to focus on recall as False Negatives can be extremely costly in health matters. So I gave utmost weightage to negative or 0 class which sigificantly increased the overall precision and the accuracy. What actually is happening is that because the amount of negative samples are significantly more in our data set, they are being correctly predicted because of the weightage and priority given to them thus increasing the overall accuracy however at the cost of positive sample predictions(recall) as they are less in number so the overall accuracy isnt being disturbed because of it

There was always a trade off between recall and precision

PCA(decreased accuracy) and Naive Bayes didnt work that well, it might have worked better on categorical data

The test data set was also imputed, but by not scaling the test set it increased the accuracy of our predictions

The code below was my best code till date at accuracy of 95.181%

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.model_selection import train_test_split

# Step 1: Perform a 70/30 train-test split
train_f, val_f, train_t, val_t = train_test_split(Features_reduced, Target, test_size=0.30, random_state=42)
dt = DecisionTreeClassifier(max_depth=1, max_features =6 ,random_state=42, max_leaf_nodes=21, class_weight={0: 5.5, 1: 1})

gb_model = AdaBoostClassifier(
    estimator=dt,
    n_estimators=600,   # Number of boosting stages to be used
    learning_rate=0.1

)

# Step 2: Initialize the Gradient Boosting Classifier
# gb_model = GradientBoostingClassifier(
#     n_estimators=100,   # Number of boosting stages to be used
#     learning_rate=0.2,  # Shrinks the contribution of each tree
#     max_depth=1,        # Maximum depth of the individual trees
#     random_state=42,
#     max_features=6
# )

# Step 3: Train the Boosting model
gb_model.fit(train_f, train_t)

# Step 4: Predict on the validation set
val_predictions = gb_model.predict(val_f)

# Step 5: Calculate accuracy,precision,f1 and recall
accuracy = accuracy_score(val_t, val_predictions)
recall = recall_score(val_t, val_predictions)
precision = precision_score(val_t,val_predictions)
f1 = f1_score(val_t,val_predictions)

# Output the validation accuracy and recall
print("Validation accuracy:", accuracy)
print("Validation recall:", recall)
print("precision: ",precision)
print("f1: ",f1)

# Step 6: Impute missing values in the test data
test_df[mean_impute_cols] = mean_imputer.transform(test_df[mean_impute_cols])
test_df[mode_impute_cols] = mode_imputer.transform(test_df[mode_impute_cols])

# Prepare the test set
test_features = pd.DataFrame(test_df, columns=test_df.columns)
test_features = test_features.drop(columns=['RecordId'] + dropped_columns)
#test_features = scaler.transform(test_features)

# Step 7: Make predictions on the test set
test_pred = gb_model.predict_proba(test_features)[:, 1]

# Step 8: Create the submission DataFrame
kaggle_submission = pd.DataFrame({'RecordId': test_df['RecordId'].astype(int), 'Y': test_pred})

# Step 9: Save the submission file
kaggle_submission.to_csv('ada2_boosting_submission.csv', index=False)

print('Submission file created successfully.')

# Validation accuracy: 0.997494481086718
# Validation recall: 0.09836065573770492
# precision 0.47368421052631576
# Submission file created successfully.

# Validation accuracy: 0.9973319609409916
# Validation recall: 0.11475409836065574
# precision 0.375
# Submission file created successfully. learning rate = 0.4

# Validation accuracy: 0.9975486544686268
# Validation recall: 0.10382513661202186
# precision 0.5277777777777778
# Submission file created successfully.     n_estimators=100,   # Number of boosting stages to be used
    # learning_rate=0.4,  # Shrinks the contribution of each tree
    # max_depth=1,        # Maximum depth of the individual trees
    # random_state=42,
    # max_features=6



# Validation accuracy: 0.9976299145414901
# Validation recall: 0.1092896174863388
# precision 0.625
# Submission file created successfully.
# gb_model = GradientBoostingClassifier(
    # n_estimators=100,   # Number of boosting stages to be used
    # learning_rate=0.2,  # Shrinks the contribution of each tree
    # max_depth=1,        # Maximum depth of the individual trees
    # random_state=42,
    # max_features=6
# )

    

I also tried DTs and Random forest, over there the optimal depth was 15 and optimal max features were 8, then again increasing the estimators increased the accuracy, greater than 15 depth and greater than max features used to decrease the accuracy and over fit. Lesser than 8 features used and less than 8 depth used to underfit

I then tried bagging with rf and it worked pretty well when i set the class weight to balanced, setting the class weight to balanced meant that we were focusing on predicting the 1s accuractely, rf is used usually on imbalance data sets and over here using it with bagging increased my rf accuracy from 91% to bagging with rf 93.8 %

Increasing estimators increased the overall accuracy, max features were set at 6